In [1]:
import pandas as pd
from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools

import mlrose # for travelling salesman problem

# that was fixed in mlrose_hiive, though the outputs are different
# import mlrose_hiive
import numpy as np

C:\Users\Usuario\anaconda3\lib\site-packages\sklearn\externals\six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [2]:
attractions = pd.read_excel("paris.xlsx", sheet_name='attractions')

In [3]:
hotels = pd.read_excel("paris_complete.xlsx", sheet_name='all_hotels')

In [4]:
paris = pd.concat((attractions,hotels), axis=0).reset_index()

In [5]:
paris.head(140)

,index,Place,lat,lon,Concept
0,0,Eifel Tower,48.858376,2.294485,Attraction
1,1,Louvre Museum,48.860710,2.337612,Attraction
2,2,Cathedrale Notre Dame,48.852968,2.349901,Attraction
3,3,Arc de Triomphe,48.873878,2.295069,Attraction
4,4,Les Invalides,48.856609,2.312668,Attraction
...,...,...,...,...,...
125,120,Hôtel Mercure Paris 9 Pigalle Sacré-Coeur,48.882406,2.336529,Hotel
126,121,Hôtel Mercure Paris Montmartre Sacré-Coeur,48.885370,2.330187,Hotel
127,122,Hôtel Mercure Paris Alésia,48.823774,2.324111,Hotel
128,123,Hôtel Mercure Paris Centre Tour Eiffel,48.855245,2.291911,Hotel


In [6]:
def get_distance(point1: dict, point2: dict) -> tuple:
    
    url = f"""http://routing.openstreetmap.de/routed-foot/route/v1/driving/{point1["lon"]},{point1["lat"]};{point2["lon"]},{point2["lat"]}?"""
    r = requests.get(url)
    
    # get the distance from the returned values
    route = json.loads(r.content)["routes"][0]
    return (route["distance"], route["duration"])

In [119]:
hotels['Duration'] = 'x'
hotels['Distance'] = 'x'
# Ojo que aca borre cosas

# Beginning the loop

In [16]:
lists = np.arange(5,130)
lists

array([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129])

In [123]:
# Este guardarlo
for x in lists:
    DF = paris.loc[[0,1,2,3,4,x],:].reset_index()
    
    dist_array = []
    for i , r in DF.iterrows():
        point1 = {"lat": r["lat"], "lon": r["lon"]}
        for j, o in DF[DF.index != i].iterrows():
            point2 = {"lat": o["lat"], "lon": o["lon"]}
            dist, duration = get_distance(point1, point2)
            dist_array.append((i, j, duration, dist))
            
    distances_df = pd.DataFrame(dist_array,columns=["origin","destination","duration(s)","distnace(m)"])
    
    # Travelling Tourism Problem
    
    length = DF.shape[0]
    
    dist_list = list(distances_df[["origin","destination","duration(s)"]].sort_values(by=["origin","destination"]).to_records(index=False))
        
    fitness_dists = mlrose.TravellingSales(distances = dist_array)
    
    problem_fit = mlrose.TSPOpt(length = length, fitness_fn = fitness_dists,
                            maximize=False)

    mlrose.genetic_alg(problem_fit, random_state = 2)
    
    best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)
    
    # print(f"The best state found is: {best_state}, taking {best_fitness} ({str(datetime.timedelta(seconds=best_fitness))})")
    
    orders = {attraction: order for order, attraction in enumerate(best_state)}
    
    DF["order"] = DF.index.map(orders)
    DF = DF.sort_values(by="order",ascending=True)
    DF = DF.append(DF)
    
    DF["First_Attraction"] = DF["Place"].shift(-1, fill_value= "test")
    DF["Second_Attraction"] = DF["Place"].shift(-2, fill_value= "test")
    DF["Third_Attraction"] = DF["Place"].shift(-3, fill_value= "test")
    DF["Fourth_Attraction"] = DF["Place"].shift(-4, fill_value= "test")
    DF["Last_Attraction"] = DF["Place"].shift(-5, fill_value= "test")
    DF = DF.head(6)

    distances_df_b = distances_df.merge(DF[["Place"]], left_on = "origin", right_index=True).rename(columns={"Place":"origin_name"})
    distances_df_b = distances_df_b.merge(DF[["Place"]], left_on = "destination", right_index=True).rename(columns={"Place":"destination_name"})

    ordered_DF = DF.merge(distances_df_b[["origin_name","destination_name","duration(s)", "distnace(m)"]], left_on=["Place","First_Attraction"], right_on=["origin_name","destination_name"], how="left")
    
    DF.loc[5, 'Place']
    next_previous = ordered_DF[ordered_DF['Place']==DF.loc[5, 'Place']]
    next_previous.reset_index(drop=True)
    next_previous.reset_index(drop=True, inplace=True)
    
    hotels.loc[x-5,'Duration'] = ordered_DF['duration(s)'].sum()
    hotels.loc[x-5,'Distance'] = ordered_DF['distnace(m)'].sum()
    hotels.loc[x-5,'First_Attraction'] = next_previous.loc[0,'First_Attraction']
    hotels.loc[x-5,"Second_Attraction"] = next_previous.loc[0,'Second_Attraction']
    hotels.loc[x-5,"Third_Attraction"] = next_previous.loc[0,'Third_Attraction']
    hotels.loc[x-5,"Fourth_Attraction"] = next_previous.loc[0,'Fourth_Attraction']
    hotels.loc[x-5,'Last_Attraction'] = next_previous.loc[0,'Last_Attraction']

In [128]:
hotels.head(5)

,Place,lat,lon,Concept,Duration,Distance,First_Attraction,Second_Attraction,Third_Attraction,Fourth_Attraction,Last_Attraction
0,Novotel Paris 13 Porte d'Italie,48.815989,2.364190,Hotel,17862.7,22161.3,Les Invalides,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame
1,Novotel Paris 14 Porte d'Orléans,48.819548,2.326132,Hotel,16393.5,20362,Les Invalides,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame
2,Novotel Paris Sud Porte de Charenton﻿,48.826443,2.405103,Hotel,19811.5,24624.6,Louvre Museum,Arc de Triomphe,Eifel Tower,Les Invalides,Cathedrale Notre Dame
3,Novotel Paris Pont de Sèvres,48.826851,2.221248,Hotel,21704.1,26961.1,Eifel Tower,Les Invalides,Cathedrale Notre Dame,Louvre Museum,Arc de Triomphe
4,Novotel Suites Paris Expo Porte De Versailles,48.827606,2.304598,Hotel,15451,19158.9,Eifel Tower,Arc de Triomphe,Les Invalides,Louvre Museum,Cathedrale Notre Dame


In [ ]:
#hotels_appended = hotels.append(attractions)
#hotels_appended.to_csv('hotels_appended.csv')
#hotels_appended.to_csv('hotels_appended.csv')
#hotels_merged = hotels_appended.merge(hotel_details, how = 'left', left_on='Place', right_on='Hotel')
#hotels_merged.to_excel('hotels_merged.xlsx')

In [132]:
hotel_details = pd.read_excel("paris_complete.xlsx", sheet_name='details')

In [133]:
hotels_merged = hotels.merge(hotel_details, how = 'left', left_on='Place', right_on='Hotel')

In [134]:
hotels_merged

,Place,lat,lon,Concept,Duration,Distance,First_Attraction,Second_Attraction,Third_Attraction,Fourth_Attraction,Last_Attraction,Hotel,Brand,Stars,Price
0,Novotel Paris 13 Porte d'Italie,48.815989,2.364190,Hotel,17862.7,22161.3,Les Invalides,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame,Novotel Paris 13 Porte d'Italie,Novotel,4,135
1,Novotel Paris 14 Porte d'Orléans,48.819548,2.326132,Hotel,16393.5,20362,Les Invalides,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame,Novotel Paris 14 Porte d'Orléans,Novotel,4,156
2,Novotel Paris Sud Porte de Charenton﻿,48.826443,2.405103,Hotel,19811.5,24624.6,Louvre Museum,Arc de Triomphe,Eifel Tower,Les Invalides,Cathedrale Notre Dame,Novotel Paris Sud Porte de Charenton﻿,Novotel,4,135
3,Novotel Paris Pont de Sèvres,48.826851,2.221248,Hotel,21704.1,26961.1,Eifel Tower,Les Invalides,Cathedrale Notre Dame,Louvre Museum,Arc de Triomphe,Novotel Paris Pont de Sèvres,Novotel,4,192
4,Novotel Suites Paris Expo Porte De Versailles,48.827606,2.304598,Hotel,15451,19158.9,Eifel Tower,Arc de Triomphe,Les Invalides,Louvre Museum,Cathedrale Notre Dame,Novotel Suites Paris Expo Porte De Versailles,Novotel,4,212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Hôtel Mercure Paris 9 Pigalle Sacré-Coeur,48.882406,2.336529,Hotel,12735,15785.4,Arc de Triomphe,Eifel Tower,Les Invalides,Cathedrale Notre Dame,Louvre Museum,Hôtel Mercure Paris 9 Pigalle Sacré-Coeur,Mercure,4,169
121,Hôtel Mercure Paris Montmartre Sacré-Coeur,48.885370,2.330187,Hotel,12887.5,15981.5,Arc de Triomphe,Eifel Tower,Les Invalides,Cathedrale Notre Dame,Louvre Museum,Hôtel Mercure Paris Montmartre Sacré-Coeur,Mercure,4,224
122,Hôtel Mercure Paris Alésia,48.823774,2.324111,Hotel,15707.8,19524.2,Les Invalides,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame,Hôtel Mercure Paris Alésia,Mercure,4,135
123,Hôtel Mercure Paris Centre Tour Eiffel,48.855245,2.291911,Hotel,11199.4,13900.8,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame,Les Invalides,Hôtel Mercure Paris Centre Tour Eiffel,Mercure,4,255


In [135]:
hotels_merged.to_excel('hotels_merged.xlsx')

In [7]:
hotels_merged = pd.read_excel('hotels_merged.xlsx')

In [22]:
hotels_attractions_merged = hotels_merged.append(attractions)
hotels_attractions_merged.head(4)

,Unnamed: 0,Place,lat,lon,Concept,Duration,Distance,First_Attraction,Second_Attraction,Third_Attraction,Fourth_Attraction,Last_Attraction,Hotel,Brand,Stars,Price
0,0.0,Novotel Paris 13 Porte d'Italie,48.815989,2.364190,Hotel,17862.7,22161.3,Les Invalides,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame,Novotel Paris 13 Porte d'Italie,Novotel,4.0,135.0
1,1.0,Novotel Paris 14 Porte d'Orléans,48.819548,2.326132,Hotel,16393.5,20362.0,Les Invalides,Eifel Tower,Arc de Triomphe,Louvre Museum,Cathedrale Notre Dame,Novotel Paris 14 Porte d'Orléans,Novotel,4.0,156.0
2,2.0,Novotel Paris Sud Porte de Charenton﻿,48.826443,2.405103,Hotel,19811.5,24624.6,Louvre Museum,Arc de Triomphe,Eifel Tower,Les Invalides,Cathedrale Notre Dame,Novotel Paris Sud Porte de Charenton﻿,Novotel,4.0,135.0
3,3.0,Novotel Paris Pont de Sèvres,48.826851,2.221248,Hotel,21704.1,26961.1,Eifel Tower,Les Invalides,Cathedrale Notre Dame,Louvre Museum,Arc de Triomphe,Novotel Paris Pont de Sèvres,Novotel,4.0,192.0


In [23]:
hotels_attractions_merged.to_excel('hotels_attractions_merged.xlsx')

# Extract Distances Tables

In [103]:
lists = np.arange(5,130)
lists

array([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,
        44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,
        83,  84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129])

In [104]:
distances_extract = pd.DataFrame()

In [105]:
for x in lists:
    DF = paris.loc[[0,1,2,3,4,x],:]
    
    dist_array = []
    for i , r in DF.iterrows():
        point1 = {"lat": r["lat"], "lon": r["lon"]}
        for j, o in DF[DF.index != i].iterrows():
            point2 = {"lat": o["lat"], "lon": o["lon"]}
            dist, duration = get_distance(point1, point2)
            dist_array.append((i, j, duration, dist))
            
    distances_temp = pd.DataFrame(dist_array,columns=["origin","destination","duration(s)","distnace(m)"])
    distances_extract = distances_extract.append(distances_temp)

In [188]:
distances_extract_b = distances_extract.groupby(['origin','destination']).max()

In [189]:
distances_extract_b = distances_extract_b.merge(paris_reseted[["Place"]],left_on = "origin",right_index=True).rename(columns={"Place":"origin_name"})
distances_extract_b = distances_extract_b.merge(paris_reseted[["Place"]],left_on = "destination",right_index=True).rename(columns={"Place":"destination_name"})

In [190]:
distances_extract_b = distances_extract_b[['origin_name', 'destination_name', 'duration(s)','distnace(m)']]
distances_extract_b['Concat'] = distances_extract_b['origin_name'] + '-' + distances_extract_b['destination_name']

In [192]:
distances_extract_b.to_excel('distances_extract.xlsx')

In [202]:
distances_extract_b

,,origin_name,destination_name,duration(s),distnace(m),Concat
origin,destination,,,,,
0,1,Eifel Tower,Louvre Museum,3111.6,3854.7,Eifel Tower-Louvre Museum
2,1,Cathedrale Notre Dame,Louvre Museum,1314.8,1641.5,Cathedrale Notre Dame-Louvre Museum
3,1,Arc de Triomphe,Louvre Museum,3021.8,3771.9,Arc de Triomphe-Louvre Museum
4,1,Les Invalides,Louvre Museum,2245.4,2782.9,Les Invalides-Louvre Museum
5,1,Novotel Paris 13 Porte d'Italie,Louvre Museum,5235.8,6512.0,Novotel Paris 13 Porte d'Italie-Louvre Museum
...,...,...,...,...,...,...
125,0,Hôtel Mercure Paris 9 Pigalle Sacré-Coeur,Eifel Tower,4124.5,5119.2,Hôtel Mercure Paris 9 Pigalle Sacré-Coeur-Eife...
126,0,Hôtel Mercure Paris Montmartre Sacré-Coeur,Eifel Tower,4153.1,5147.2,Hôtel Mercure Paris Montmartre Sacré-Coeur-Eif...
127,0,Hôtel Mercure Paris Alésia,Eifel Tower,4236.3,5250.9,Hôtel Mercure Paris Alésia-Eifel Tower


# Consolidating all in one table

In [273]:
hotels_attractions = pd.read_excel('hotels_attractions_merged.xlsx')

In [288]:
hotels_attractions.tail(3)

,Unnamed: 0,Unnamed: 0.1,Place,lat,lon,Concept,Duration,Distance,First_Attraction,Second_Attraction,...,Second_Walk_duration(s),Second_Walk_distnace(m),Third_Walk_duration(s),Third_Walk_distnace(m),Fourth_Walk_duration(s),Fourth_Walk_distnace(m),Fifth_Walk_duration(s),Fifth_Walk_distnace(m),Sixth_Walk_duration(s),Sixth_Walk_distnace(m)
127,2,NaN,Cathedrale Notre Dame,48.852968,2.349901,Attraction,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,3,NaN,Arc de Triomphe,48.873878,2.295069,Attraction,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,4,NaN,Les Invalides,48.856609,2.312668,Attraction,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [275]:
hotels_attractions['First_Walk'] = hotels_attractions['Place']+"-"+hotels_attractions['First_Attraction']
hotels_attractions['Second_Walk'] = hotels_attractions['First_Attraction']+"-"+hotels_attractions['Second_Attraction']
hotels_attractions['Third_Walk'] = hotels_attractions['Second_Attraction']+"-"+hotels_attractions['Third_Attraction']
hotels_attractions['Fourth_Walk'] = hotels_attractions['Third_Attraction']+"-"+hotels_attractions['Fourth_Attraction']
hotels_attractions['Fifth_Walk'] = hotels_attractions['Fourth_Attraction']+"-"+hotels_attractions['Last_Attraction']
hotels_attractions['Sixth_Walk'] = hotels_attractions['Last_Attraction']+"-"+hotels_attractions['Place']

In [276]:
distances_extract_b = distances_extract_b[['Concat','duration(s)','distnace(m)']]

Merge First Walk

In [277]:
hotels_attractions = hotels_attractions.merge(distances_extract_b, how = 'left', left_on ='First_Walk', right_on='Concat')   

In [278]:
hotels_attractions = hotels_attractions.drop('Concat', axis=1)

In [279]:
hotels_attractions = hotels_attractions.rename(columns={"duration(s)":"First_Walk_duration(s)"})
hotels_attractions = hotels_attractions.rename(columns={"distnace(m)":"First_Walk_distnace(m)"})

Merge Second Walk

In [280]:
hotels_attractions = hotels_attractions.merge(distances_extract_b, how = 'left', left_on ='Second_Walk', right_on='Concat')   

In [281]:
hotels_attractions = hotels_attractions.drop('Concat', axis=1)

In [282]:
hotels_attractions = hotels_attractions.rename(columns={"duration(s)":"Second_Walk_duration(s)"})
hotels_attractions = hotels_attractions.rename(columns={"distnace(m)":"Second_Walk_distnace(m)"})

Merge Third Walk

In [283]:
hotels_attractions = hotels_attractions.merge(distances_extract_b, how = 'left', left_on ='Third_Walk', right_on='Concat')   
hotels_attractions = hotels_attractions.drop('Concat', axis=1)
hotels_attractions = hotels_attractions.rename(columns={"duration(s)":"Third_Walk_duration(s)"})
hotels_attractions = hotels_attractions.rename(columns={"distnace(m)":"Third_Walk_distnace(m)"})

Merge Fourth Walk

In [284]:
hotels_attractions = hotels_attractions.merge(distances_extract_b, how = 'left', left_on ='Fourth_Walk', right_on='Concat')   
hotels_attractions = hotels_attractions.drop('Concat', axis=1)
hotels_attractions = hotels_attractions.rename(columns={"duration(s)":"Fourth_Walk_duration(s)"})
hotels_attractions = hotels_attractions.rename(columns={"distnace(m)":"Fourth_Walk_distnace(m)"})

Merge Fifth Walk

In [285]:
hotels_attractions = hotels_attractions.merge(distances_extract_b, how = 'left', left_on ='Fifth_Walk', right_on='Concat')   
hotels_attractions = hotels_attractions.drop('Concat', axis=1)
hotels_attractions = hotels_attractions.rename(columns={"duration(s)":"Fifth_Walk_duration(s)"})
hotels_attractions = hotels_attractions.rename(columns={"distnace(m)":"Fifth_Walk_distnace(m)"})

Merge Sixth Walk

In [286]:
hotels_attractions = hotels_attractions.merge(distances_extract_b, how = 'left', left_on ='Sixth_Walk', right_on='Concat')   
hotels_attractions = hotels_attractions.drop('Concat', axis=1)
hotels_attractions = hotels_attractions.rename(columns={"duration(s)":"Sixth_Walk_duration(s)"})
hotels_attractions = hotels_attractions.rename(columns={"distnace(m)":"Sixth_Walk_distnace(m)"})

In [287]:
hotels_attractions.to_excel('hotels_attractions_merged_with_distances.xlsx')

# Drawing

In [34]:
import requests
import folium
import polyline

In [35]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://routing.openstreetmap.de/routed-foot/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [38]:
test_route=(48.858376, 2.294485, 48.860710, 2.337612)


In [46]:
pickup_lon, pickup_lat, dropoff_lon, dropoff_lat = 2.294485,48.858376, 2.337612, 48.860710 
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
test_route

{'route': [(48.85834, 2.29454),
  (48.85859, 2.29581),
  (48.85878, 2.29614),
  (48.85948, 2.29595),
  (48.85991, 2.2966),
  (48.86029, 2.29783),
  (48.86047, 2.2977),
  (48.86085, 2.29812),
  (48.86139, 2.29762),
  (48.8615, 2.29779),
  (48.86167, 2.29765),
  (48.86204, 2.29897),
  (48.8622, 2.29889),
  (48.86264, 2.29979),
  (48.86287, 2.30148),
  (48.86287, 2.30607),
  (48.8632, 2.31789),
  (48.86129, 2.32433),
  (48.86254, 2.32535),
  (48.86097, 2.33025),
  (48.86126, 2.33047),
  (48.86091, 2.33182),
  (48.86097, 2.33201),
  (48.86061, 2.33376),
  (48.86099, 2.33399),
  (48.86017, 2.33713),
  (48.86064, 2.33738),
  (48.8606, 2.33755)],
 'start_point': [48.858339, 2.294543],
 'end_point': [48.860601, 2.337551],
 'distance': 3854.7}

In [47]:
def get_map(route):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=13)

    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m

In [48]:
get_map(test_route)